In [1]:
# !nvcc --version
# !nvidia-smi

import os, shutil
import numpy as np
import matplotlib.pyplot as plt
from cellpose import core, utils, io, models, metrics
from data_loader import CellDataset, split_dataset
from cellpose import models

use_GPU = core.use_gpu()
yn = ['NO', 'YES']
print(f'>>> GPU activated? {yn[use_GPU]}')

In [ ]:
initial_model = "cyto" #@param ['cyto','nuclei','tissuenet','livecell','cyto2','CP','CPx','TN1','TN2','TN3','LC1','LC2','LC3','LC4','scratch']
model_dir = "./"
model_name = "single_cell_params_test.pt" #@param {type:"string"}
n_epochs =  100#@param {type:"number"}
learning_rate = 0.01 
weight_decay = 0.0001
batch_size = 8
chan = 0
chan2 = 0

Default advanced parameters enabled


In [3]:
data_paths = []
with open("./file_paths.txt", "r") as fp:
    data_paths = fp.read().strip().split('\n')

In [4]:
train_paths, test_paths = split_dataset(data_paths, 0.2)
train_ds = CellDataset(paths=train_paths, generate_flows=True)
test_ds = CellDataset(paths=test_paths, generate_flows=True)

In [5]:
# start logger (to see training across epochs)
logger = io.logger_setup()

# DEFINE CELLPOSE MODEL (without size model)
model = models.CellposeModel(gpu=use_GPU)

# set channels
channels = [chan, chan2]

new_model_path = model.train(train_ds, test_ds, save_path=model_dir, 
                              n_epochs=batch_size,
                              learning_rate=learning_rate, 
                              weight_decay=weight_decay, 
                              nimg_per_epoch=5,
                              model_name=model_name,
                              batch_size=batch_size
                              )

# diameter of labels in training images
diam_labels = model.diam_labels.copy()

2023-08-12 21:45:36,099 [INFO] WRITING LOG OUTPUT TO /home/balint/.cellpose/run.log
2023-08-12 21:45:36,099 [INFO] 
cellpose version: 	2.2 
platform:       	linux 
python version: 	3.8.15 
torch version:  	2.0.0+cu117
2023-08-12 21:45:36,101 [INFO] ** TORCH CUDA version installed and working. **
2023-08-12 21:45:36,102 [INFO] >>>> using GPU
2023-08-12 21:48:50,276 [INFO] >>>> median diameter set to = 45
2023-08-12 21:48:50,277 [INFO] >>>> mean of training label mask diameters (saved to model) 65.097
2023-08-12 21:48:50,291 [INFO] >>>> training network with 2 channel input <<<<
2023-08-12 21:48:50,295 [INFO] >>>> LR: 0.00100, batch_size: 8, weight_decay: 0.00010
2023-08-12 21:48:50,296 [INFO] >>>> ntrain = 195, ntest = 65
2023-08-12 21:48:50,298 [INFO] >>>> nimg_per_epoch = 195
2023-08-12 21:54:02,255 [INFO] Epoch 0/1:: 100%|##########| 9/9 [01:23<00:00,  9.29s/it]
2023-08-12 21:54:02,256 [INFO] Epoch 0, Time 312.0s, Loss 0.8324, Loss Test 0.8536, LR 0.0000
2023-08-12 21:54:02,262 [INFO

## Evaluate on test data (optional)

If you have test data, check performance

In [6]:
# get files (during training, test_data is transformed so we will load it again)
output = io.load_train_test_data(test_dir, mask_filter='_seg.npy')
test_data, test_labels = output[:2]

# run model on test images
masks = model.eval(test_data, 
                   channels=[chan, chan2],
                   diameter=diam_labels)[0]

# check performance using ground truth labels
ap = metrics.average_precision(test_labels, masks)[0]
print('')
print(f'>>> average precision at iou threshold 0.5 = {ap[:,0].mean():.3f}')


TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

plot masks

In [ ]:

plt.figure(figsize=(12,8), dpi=150)
for k,im in enumerate(test_data):
    img = im.copy()
    plt.subplot(3,len(train_files), k+1)
    img = np.vstack((img, np.zeros_like(img)[:1]))
    img = img.transpose(1,2,0)
    plt.imshow(img)
    plt.axis('off')
    if k==0:
        plt.title('image')

    plt.subplot(3,len(train_files), len(train_files) + k+1)
    plt.imshow(masks[k])
    plt.axis('off')
    if k==0:
        plt.title('predicted labels')

    plt.subplot(3,len(train_files), 2*len(train_files) + k+1)
    plt.imshow(test_labels[k])
    plt.axis('off')
    if k==0:
        plt.title('true labels')
plt.tight_layout()